In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from CLEARview import lattice, aperture, elements, opticsUtils, plot
import time
import pandas as pd

In [2]:
# Relativistic parametres 
P0 = 200.0                    #[MeV] 
m0c2 = 511e-03                #[MeV]
E0 = np.sqrt(m0c2**2 + P0**2) #[MeV]

# Initial twiss parametres 
N = 200         # Number of particles
epsn_x = 10.0   #[um]
epsn_y = 10.0   #[um]

# Find geometrical emittance
epsg_x = opticsUtils.get_eps_g(epsn_x,E0) 
epsg_y = opticsUtils.get_eps_g(epsn_y,E0) 

# Constraints in Plasma lens
#sigma_PL = 0.2e-03     #[m]
alpha_PL = 0.0

# Initial beta and alpha in the position of the Plasma lens
beta0x = 0.5002
alpha0x = 0.02
beta0y = 0.5002
alpha0y = 0.02

x0 = 0.0  #[mm]
y0 = 0.0  #[1e-3]
xp0 = 0.0 #[mm]
yp0 = 0.0 #[1e-3]

In [3]:
# Find the Twiss parametres from a distribution in case needed

def Twiss(x,xp): 
    cov = np.cov(x, xp)
    erms = np.sqrt(np.linalg.det(cov))
    beta = cov[0,0]/erms
    alfa = -cov[0,1]/erms
    gamma = cov[1,1]/erms
    return cov, erms, beta, alfa, gamma

In [4]:
from CLEARview import clearST_survey0_tfs #get the lattice

def lattice(focus,start,end='MTV_DUMP'):
    # Fix the setup of the quadrupoles
    latticeCLEAR = clearST_survey0_tfs.buildLattice(P0, doFinalize=False)
    sPLE = latticeCLEAR.getElemS('PLE0800M') # Get the position of the middle of the plasmalens
    latticeCLEAR.implicitSeq_removeElem('CA.DHJ0780') # Remove kicker magnet from the setup
    latticeCLEAR.implicitSeq_removeElem('BTV0800') # Remove to avoid overlapping elements when adding PLE
    latticeCLEAR.implicitSeq_removeElem('PLE0800M') # Remove the marker for the plasmalens
    latticeCLEAR.implicitSeq_removeElem('QS0760') # Remove kicker magnet from the setup
    latticeCLEAR.implicitSeq_removeElem('CA.DHJ0840') # Remove kicker magnet from the setup
    latticeCLEAR.implicitSeq_addElem(elements.CLEAR_Q(0.0, P0), 'QFD0860', 16.2625, refer='end', referFrom=None) # Add focusing magnet
    #latticeCLEAR.implicitSeq_addElem(elements.Marker(), 'Symmetry point', 18.1815)
    #latticeCLEAR.implicitSeq_addElem(elements.Marker(),'P', 15.007499999999999) #Mark the start of the plasmalens  
    #latticeCLEAR.implicitSeq_addElem(elements.Marker(), 'Start', 14.400)

    # Remove some markers for a simpler setup (easier to read the plot)
    #latticeCLEAR.implicitSeq_removeElem('CA.QS0760')
    latticeCLEAR.implicitSeq_removeElem('CA.QS0870')
    latticeCLEAR.implicitSeq_removeElem('CA.BPM0890')
    latticeCLEAR.implicitSeq_removeElem('CA.CLEAR$END')
    latticeCLEAR.implicitSeq_removeElem('CA.CLDUMP$START')
    latticeCLEAR.implicitSeq_removeElem('CA.VAW0915')
    latticeCLEAR.implicitSeq_removeElem('CA.ICT0930')
    latticeCLEAR.implicitSeq_removeElem('CLDUMP$END')
    latticeCLEAR.implicitSeq_removeElem('CA.BTV0910') # Fjernet pga jeg får trøbbel med optimeringen 
    #latticeCLEAR.implicitSeq_finalize()

    # Add markers for different focal points
    #latticeCLEAR.implicitSeq_addElem(elements.Marker(), 'Focus point', 18.1815) # Dette er fokuspunktet som er symmetrisk
    latticeCLEAR.implicitSeq_addElem(elements.Marker(), 'Focus point', focus)
    #latticeCLEAR.implicitSeq_finalize()

    # ADD PLASMALENS
    latticeCLEAR.implicitSeq_addElem( elements.CLEAR_PlasmaLens( 20e-3, 0.50, 0.0, P0 ), 'PLE0800', sPLE)

    latticeCLEAR.implicitSeq_finalize()

    latticeCLEAR_PLE = latticeCLEAR.getSubLattice(start, end)
    
    return latticeCLEAR_PLE

lol = lattice(18.1815,'BTV0730', 'PLE0800')

In [ ]:
focus0 = 18.1815

def minFun_opt(xin):
        I5_, I6_, I7_ = xin
        
        beta0x = 0.5002
        beta0y = 0.5002
        alpha0x = 0.02
        alpha0y = 0.02
        
        latticeCLEAR_PLE = lattice(focus0,starting_point, end_point)
        focus_Index0 = latticeCLEAR_PLE.getTwissIdx("BTV0730")
        
        latticeCLEAR_PLE.elements['QFD0760'].update(I5_, P0)
        latticeCLEAR_PLE.elements['QDD0765'].update(-I6_, P0)
        latticeCLEAR_PLE.elements['QFD0770'].update(I7_, P0)

        (XM,YM) = latticeCLEAR_PLE.getTwissMatrix(firstElem=None,lastElem="PLE0800",lastElemExclusive=False)
        print('XM:', XM)
        print('YM:', YM)
        #print("XM:", XM, "YM:", YM)
        #(XM,YM) = latticeCLEAR_PLE.getTwissMatrix(firstElem="CA.PLE0800",lastElem="CA.QS0760",lastElemExclusive=True)

        #-----------------------------------------------------------------------------------
        # Twiss matrix for propagating
        xt_init = np.asarray([beta0x, alpha0x, opticsUtils.getGamma(alpha0x,beta0x)])
        yt_init = np.asarray([beta0y, alpha0y, opticsUtils.getGamma(alpha0y,beta0y)])
        #-----------------------------------------------------------------------------------

        xt_init = np.linalg.solve(XM,xt_init)
        yt_init = np.linalg.solve(YM,yt_init)
        

        (s, xt, yt, xTra, yTra) = latticeCLEAR_PLE.computeTwiss(xt_init, yt_init)
        
        opticsUtils.printLatticeOptics(latticeCLEAR_PLE, s, xt, yt, epsg_x,epsg_y, xTra, yTra)
        
        alphax = xt[1, focus_Index0]
        alphay = yt[1, focus_Index0]
        
        alpha_PLE = 0.02
        beta_PLE = 0.5002
        
        betax = xt[0, focus_Index0]
        betay = yt[0, focus_Index0]
 
        #return (alphax**2 + alphay**2)
        return (alphax**2 + alphay**2)*100 + (np.sqrt(betax/betay)-1)**2 
        #return (betax-beta_PLE)**2  + (betay-beta_PLE)**2  + (alphax-alpha_PLE)**2+ (alphay-alpha_PLE)**2

latticeCLEAR_PLE = lattice(focus0, starting_point, end_point)
focus_Index0 = latticeCLEAR_PLE.getTwissIdx("BTV0730")

I_final = [60.70, 97.56, 45.53] # SCALED CURRENTS FROM CLEAR EXPERIMENT 19.09.2024 (original currents belong to P0=184.5MeV)

bounds = [(0, 500), (0, 500), (0, 500)]

optRes_final = minimize(minFun_opt, x0=I_final, bounds=bounds) 
print(optRes_final)
I5_opt_final = optRes_final.x[0]
I6_opt_final = optRes_final.x[1]
I7_opt_final = optRes_final.x[2]
print(f"First optimization at PLE gives:")
print(f"I1 = {I5_opt_final} [A],  I2 = {I6_opt_final} [A], I3 = {I7_opt_final} [A]")

Last print from optimization contains the correct Twiss parametres, these are then used for further visualizations of beam propagations through lattice including the matching section